In [14]:
! pip install mysql-connector-python
! pip install numpy
! pip install scikit-learn
! pip install python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [5]:
import mysql.connector
#import dotenv and use it to load the environment variables
from dotenv import load_dotenv
load_dotenv()
import os
# Connect to MySQL
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_DATABASE")
db_user = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")

# Connect to MySQL using the loaded environment variables
db = mysql.connector.connect(
    host=db_host,
    port=db_port,
    database=db_name,
    user=db_user,
    password=db_password
)



In [3]:
import mysql.connector
import numpy as np
import logging
import json  # Add json to handle JSON conversion

# Setup logging to log to the notebook and a file
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

cursor = db.cursor()

# Fetch normalized vectors
cursor.execute("SELECT normalized_vector FROM vectors")

vectors = []
for idx, v in enumerate(cursor.fetchall()):
    try:
        # Log progress in Jupyter
        if idx % 1000 == 0:
            print(f"Processing vector {idx}...")

        # Remove any brackets and whitespace, then split the string by commas
        cleaned_vector = v[0].replace('[', '').replace(']', '').strip()
        vector = np.array(list(map(float, cleaned_vector.split(','))))
        vectors.append(vector)
    except ValueError as e:
        print(f"Error converting vector at index {idx}: {v[0]}")
        logging.error(f"Error converting vector at index {idx}: {v[0]}")
        logging.exception(e)

# Convert to a NumPy array
vectors = np.array(vectors)

# Log the number of valid vectors
print(f"Successfully fetched and parsed {len(vectors)} vectors.")
logging.info(f"Successfully fetched and parsed {len(vectors)} vectors.")

# Run k-means
print("Starting K-means clustering...")
num_centroids = 200
kmeans = KMeans(n_clusters=num_centroids, max_iter=100, random_state=0)
kmeans.fit(vectors)

# Log centroids creation completion
print(f"K-means clustering completed. Found {len(kmeans.cluster_centers_)} centroids.")
logging.info(f"K-means clustering completed. Found {len(kmeans.cluster_centers_)} centroids.")

# Save centroids back to the database
centroids = kmeans.cluster_centers_
for index, centroid in enumerate(centroids):
    centroid_json = json.dumps(centroid.tolist())  # Convert the NumPy array to a list and then to JSON
    
    # First, check if the centroid with the given ID exists
    cursor.execute("SELECT id FROM centroids WHERE id = %s", (index + 1,))
    result = cursor.fetchone()

    if result:
        # If the centroid exists, update it
        cursor.execute(
            "UPDATE centroids SET vector = %s WHERE id = %s", 
            (centroid_json, index + 1)
        )
        logging.info(f"Updated Centroid {index + 1} in the database.")
    else:
        # If the centroid doesn't exist, insert it
        cursor.execute(
            "INSERT INTO centroids (id, vector) VALUES (%s, %s)", 
            (index + 1, centroid_json)
        )
        logging.info(f"Inserted new Centroid {index + 1} into the database.")

db.commit()
cursor.close()
db.close()

print("Centroid processing completed successfully.")
logging.info("Centroid processing completed successfully.")


KeyboardInterrupt: 

In [6]:
import mysql.connector
import numpy as np
import logging
import json

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Assuming db connection is already established
cursor = db.cursor()

# Fetch centroids from the database
cursor.execute("SELECT id, vector FROM centroids")
centroids = cursor.fetchall()

# Convert centroids into NumPy arrays
centroid_ids = [c[0] for c in centroids]
centroid_vectors = np.array([np.array(json.loads(c[1])) for c in centroids])

# Fetch all vectors in memory
cursor.execute("SELECT id, normalized_vector FROM vectors")
vectors = cursor.fetchall()

# Process vectors in chunks
chunk_size = 100
total_vectors = len(vectors)
logging.info(f"Total vectors to process: {total_vectors}")

processed_count = 0
updates = []  # Collect updates to perform bulk updates

# Normalize centroids (if not already normalized in your data)
centroid_magnitudes = np.linalg.norm(centroid_vectors, axis=1)

for i in range(0, total_vectors, chunk_size):
    chunk = vectors[i:i + chunk_size]

    # Extract vector data and convert to NumPy
    vector_ids = [v[0] for v in chunk]
    vector_data = np.array([np.array(json.loads(v[1])) for v in chunk])

    # Normalize vectors (assuming normalized_vector is not already normalized)
    vector_magnitudes = np.linalg.norm(vector_data, axis=1)

    # Perform bulk cosine similarity calculation
    dot_products = np.dot(vector_data, centroid_vectors.T)
    cosine_similarities = dot_products / (vector_magnitudes[:, None] * centroid_magnitudes)

    # Get the index of the closest centroid for each vector
    best_matches = np.argmax(cosine_similarities, axis=1)
    
    # Prepare updates for batch execution
    for idx, best_match_idx in enumerate(best_matches):
        best_centroid_id = centroid_ids[best_match_idx]
        vector_id = vector_ids[idx]
        updates.append((best_centroid_id, vector_id))

    processed_count += len(chunk)
    logging.info(f"Processed {processed_count} out of {total_vectors} vectors")

    # Perform bulk updates in batches of 100
    if len(updates) >= 100:
        cursor.executemany(
            "UPDATE grant_vector SET centroid_id = %s WHERE vector_id = %s",
            updates
        )
        db.commit()
        updates.clear()

# Perform remaining updates if any
if updates:
    cursor.executemany(
        "UPDATE grant_vector SET centroid_id = %s WHERE vector_id = %s",
        updates
    )
    db.commit()

# Commit the changes and close the connection
cursor.close()
db.close()

logging.info("Centroid assignment for vectors completed.")


2024-10-03 22:38:08,656 - INFO - Total vectors to process: 78389
2024-10-03 22:38:08,664 - INFO - Processed 100 out of 78389 vectors
2024-10-03 22:38:30,908 - INFO - Processed 200 out of 78389 vectors
2024-10-03 22:38:39,378 - INFO - Processed 300 out of 78389 vectors
2024-10-03 22:38:47,736 - INFO - Processed 400 out of 78389 vectors
2024-10-03 22:38:56,133 - INFO - Processed 500 out of 78389 vectors
2024-10-03 22:39:04,515 - INFO - Processed 600 out of 78389 vectors
2024-10-03 22:39:12,889 - INFO - Processed 700 out of 78389 vectors
2024-10-03 22:39:21,293 - INFO - Processed 800 out of 78389 vectors
2024-10-03 22:39:29,662 - INFO - Processed 900 out of 78389 vectors
2024-10-03 22:39:38,249 - INFO - Processed 1000 out of 78389 vectors
2024-10-03 22:39:46,619 - INFO - Processed 1100 out of 78389 vectors
2024-10-03 22:39:54,964 - INFO - Processed 1200 out of 78389 vectors
2024-10-03 22:40:03,281 - INFO - Processed 1300 out of 78389 vectors
2024-10-03 22:40:11,600 - INFO - Processed 1400

KeyboardInterrupt: 

In [7]:
import mysql.connector
import numpy as np
import logging
import json

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Assuming db connection is already established
cursor = db.cursor()

# Fetch centroids from the database
cursor.execute("SELECT id, vector FROM centroids")
centroids = cursor.fetchall()

# Convert centroids into NumPy arrays
centroid_ids = [c[0] for c in centroids]
centroid_vectors = np.array([np.array(json.loads(c[1])) for c in centroids])

# Fetch all vectors in memory
cursor.execute("SELECT id, normalized_vector FROM vectors")
vectors = cursor.fetchall()

# Process vectors in chunks
chunk_size = 500
total_vectors = len(vectors)
logging.info(f"Total vectors to process: {total_vectors}")

processed_count = 0
updates = {}  # Dictionary to collect vector_id to centroid_id mappings

# Normalize centroids (if not already normalized in your data)
centroid_magnitudes = np.linalg.norm(centroid_vectors, axis=1)

for i in range(0, total_vectors, chunk_size):
    chunk = vectors[i:i + chunk_size]

    # Extract vector data and convert to NumPy
    vector_ids = [v[0] for v in chunk]
    vector_data = np.array([np.array(json.loads(v[1])) for v in chunk])

    # Normalize vectors (assuming normalized_vector is not already normalized)
    vector_magnitudes = np.linalg.norm(vector_data, axis=1)

    # Perform bulk cosine similarity calculation
    dot_products = np.dot(vector_data, centroid_vectors.T)
    cosine_similarities = dot_products / (vector_magnitudes[:, None] * centroid_magnitudes)

    # Get the index of the closest centroid for each vector
    best_matches = np.argmax(cosine_similarities, axis=1)
    
    # Collect updates for the batch
    for idx, best_match_idx in enumerate(best_matches):
        best_centroid_id = centroid_ids[best_match_idx]
        vector_id = vector_ids[idx]
        updates[vector_id] = best_centroid_id

    processed_count += len(chunk)
    logging.info(f"Processed {processed_count} out of {total_vectors} vectors")

    # Perform true batch update for each chunk
    if len(updates) >= chunk_size:
        update_query = "UPDATE grant_vector SET centroid_id = CASE "
        update_query += " ".join([f"WHEN {vector_id} THEN {centroid_id}" for vector_id, centroid_id in updates.items()])
        update_query += f" END WHERE vector_id IN ({', '.join(map(str, updates.keys()))})"
        
        cursor.execute(update_query)
        db.commit()
        updates.clear()

# Perform remaining updates if any
if updates:
    update_query = "UPDATE grant_vector SET centroid_id = CASE "
    update_query += " ".join([f"WHEN {vector_id} THEN {centroid_id}" for vector_id, centroid_id in updates.items()])
    update_query += f" END WHERE vector_id IN ({', '.join(map(str, updates.keys()))})"
    
    cursor.execute(update_query)
    db.commit()

# Commit the changes and close the connection
cursor.close()
db.close()

logging.info("Centroid assignment for vectors completed.")


2024-10-03 22:41:15,354 - INFO - Total vectors to process: 78389
2024-10-03 22:41:15,398 - INFO - Processed 500 out of 78389 vectors
2024-10-03 22:41:15,623 - INFO - Processed 1000 out of 78389 vectors
2024-10-03 22:41:15,843 - INFO - Processed 1500 out of 78389 vectors
2024-10-03 22:41:16,059 - INFO - Processed 2000 out of 78389 vectors
2024-10-03 22:41:16,271 - INFO - Processed 2500 out of 78389 vectors
2024-10-03 22:41:16,488 - INFO - Processed 3000 out of 78389 vectors
2024-10-03 22:41:16,717 - INFO - Processed 3500 out of 78389 vectors
2024-10-03 22:41:16,949 - INFO - Processed 4000 out of 78389 vectors
2024-10-03 22:41:17,172 - INFO - Processed 4500 out of 78389 vectors
2024-10-03 22:41:17,379 - INFO - Processed 5000 out of 78389 vectors
2024-10-03 22:41:17,608 - INFO - Processed 5500 out of 78389 vectors
2024-10-03 22:41:17,817 - INFO - Processed 6000 out of 78389 vectors
2024-10-03 22:41:18,044 - INFO - Processed 6500 out of 78389 vectors
2024-10-03 22:41:18,261 - INFO - Proces